# Quantization of embeddings

- Difference between x and x_hat
- Difference between dot product
- Can calculate dot product with x_hat much easier
- Growth of Delta
- 1-norm of x_hat
- Quantization error for rounding
- Quantization error for ceiling

In [1]:
from typing import Callable
import math

min_v = -1
max_v = 1
a = -128
b = 127

delta = (max_v - min_v) / (b - a)

def forward_transformation(x: float, transform: Callable[[float], int] = math.floor) -> int:
    x_tilde = a + (b - a) / (max_v - min_v) * (x - min_v)
    return transform(x_tilde)

def backward_transformation(x: int) -> float:
    return (x - a) * (max_v - min_v) / (b - a) + min_v

In [2]:
import random
import math
import numpy as np


def pnorm(x: list[float], p: int = 2) -> float:
    return math.pow(sum(np.abs(x_i) ** p for x_i in x), 1 / p)

def normalize(x: list[float]) -> list[float]:
    norm = pnorm(x, 2)
    return [x_i / norm for x_i in x]

d = 5

x = [random.random() for _ in range(d)]
y = [random.random() for _ in range(d)]

x = normalize(x)
y = normalize(y)

x_hat = [backward_transformation(forward_transformation(x_i)) for x_i in x]
y_hat = [backward_transformation(forward_transformation(y_i)) for y_i in y]

In [3]:
dot_product = np.dot(x, y)
dot_product_hat = np.dot(x_hat, y_hat)

print("<x,y> =", dot_product)
print("<x_hat,y_hat> = ", dot_product_hat)

print("Difference = ", np.abs(dot_product - dot_product_hat))
print("Error estimation = ", delta * (pnorm(x_hat, 1) + pnorm(y_hat, 1)) + d * delta ** 2)
print("Error estimation 2 = ", 2 * delta * math.sqrt(d) + d * delta ** 2)

<x,y> = 0.87700526013309
<x_hat,y_hat> =  0.8659900038446751
Difference =  0.011015256288414932
Error estimation =  0.030326797385620913
Error estimation 2 =  0.035383150127639915
